In [33]:
# pip install scikit-surprise

In [3]:
import pandas as pd
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.dataset import Dataset
from surprise.reader import Reader

In [4]:
anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")
print("anime of shape:", anime.shape)
print("rating of shape:", rating.shape)

anime of shape: (12294, 7)
rating of shape: (7813737, 3)


In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [7]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [8]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [9]:
print(f'Anime Missing Value')
print(anime.isnull().sum().sort_values(ascending=False))
print(f'\nRating Missing Values')
print(rating.isnull().sum().sort_values(ascending=False))

Anime Missing Value
rating      230
genre        62
type         25
anime_id      0
name          0
episodes      0
members       0
dtype: int64

Rating Missing Values
user_id     0
anime_id    0
rating      0
dtype: int64


#### Hadling Missing Vallues

In [10]:
# Mengisi nilai genre dan type dengan modus
anime['genre'] = anime['genre'].fillna(
anime['genre'].dropna().mode().values[0])

anime['type'] = anime['type'].fillna(
anime['type'].dropna().mode().values[0])

In [11]:
# drop columns rating pada anime
anime = anime.drop(columns=['rating'])

In [12]:
# Mengubah nilai -1 menjadi 0
rating['rating'] = rating['rating'].replace(-1, 0)

In [13]:
df = pd.merge(rating, anime, on='anime_id')
df.head()

,user_id,anime_id,rating,name,genre,type,episodes,members
0,1,20,0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
3,6,20,0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297
4,10,20,0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,683297


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7813727 entries, 0 to 7813726
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   anime_id  int64 
 2   rating    int64 
 3   name      object
 4   genre     object
 5   type      object
 6   episodes  object
 7   members   int64 
dtypes: int64(4), object(4)
memory usage: 536.5+ MB


In [15]:
df.isna().sum()

user_id     0
anime_id    0
rating      0
name        0
genre       0
type        0
episodes    0
members     0
dtype: int64

Karena -1 jika pengguna menontonnya tetapi tidak memberikan peringkat. lalu kita ubah menjadi 0, dikhawatirkan jika data tersebut drop akan membuat banyak informasi hilang

In [17]:
a = []
b = []
c = []
for x, y, z in zip(df['anime_id'], df['name'], df.index):
    if (x not in a) & (y not in b):
        a.append(x)
        b.append(y)
    if (x not in a) & (y in b):
        c.append(z)

### Check duplikat user review
Apakah ada user yang memberi review 2x pada anime yang sama? Bila ada drop salah satu row nya.

In [18]:
df[df[['user_id', 'anime_id']].duplicated()]

,user_id,anime_id,rating,name,genre,type,episodes,members
135583,42653,2001,10,Tengen Toppa Gurren Lagann,"Action, Adventure, Comedy, Mecha, Sci-Fi",TV,27,562962
647077,42653,11757,5,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,893100
804208,42653,16498,8,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,896229
965727,42653,22319,6,Tokyo Ghoul,"Action, Drama, Horror, Mystery, Psychological,...",TV,12,618056
1525017,42653,20507,9,Noragami,"Action, Adventure, Shounen, Supernatural",TV,12,515378
3440402,42653,1575,6,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25,715151
4462550,42653,23283,9,Zankyou no Terror,"Psychological, Thriller",TV,11,342893


In [19]:
# Menghapus Duplikat
df = df.drop(df[df[['user_id', 'anime_id']].duplicated()].index).reset_index(drop=True)

Membuat daftar 50 anime popular berdasarkan dengan setidaknya 500 user yang telah memberikan review

In [20]:
anime_dict1 = {}
anime_dict2 = {}
for x, y in zip(df['anime_id'], df['name']):
    if x not in anime_dict1.keys():
        anime_dict1[x] = y
        anime_dict2[y] = x

In [21]:
popular  = pd.DataFrame({'count':df.groupby('anime_id').count()['user_id'],
                       'rating':df.groupby('anime_id').mean(numeric_only=True)['rating']})

In [22]:
popular.head()

,count,rating
anime_id,,
1,15509,7.691341
5,6927,7.054425
6,11077,7.133339
7,2629,6.215291
8,413,5.368039


In [23]:
def popular_recom(k, count, dataset):
    a = []
    popular2 = dataset[dataset['count']>=count].sort_values('rating', ascending=False)
    for x in popular2.index[:k]:
        a.append(anime_dict1[x])
    return a

In [37]:
popular_recom(50, 500, popular)

['Kimi no Na wa.',
 'Ginga Eiyuu Densetsu',
 'Steins;Gate',
 'Fullmetal Alchemist: Brotherhood',
 'Gintama°',
 'Hunter x Hunter (2011)',
 'Clannad: After Story',
 'Monster',
 'Gintama',
 'Code Geass: Hangyaku no Lelouch R2',
 'Shigatsu wa Kimi no Uso',
 'Gintama&#039;',
 'Tengen Toppa Gurren Lagann',
 'Rainbow: Nisha Rokubou no Shichinin',
 'Uchuu Kyoudai',
 'Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare',
 'Mushishi',
 'Code Geass: Hangyaku no Lelouch',
 'Ping Pong The Animation',
 'Hajime no Ippo',
 'Uchuu Senkan Yamato 2199',
 'Great Teacher Onizuka',
 'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou',
 'Gintama&#039;: Enchousen',
 'Cowboy Bebop',
 'Death Note',
 'Ookami Kodomo no Ame to Yuki',
 'Cross Game',
 'Kemono no Souja Erin',
 'Yojouhan Shinwa Taikei',
 'Suzumiya Haruhi no Shoushitsu',
 'Boku dake ga Inai Machi',
 'Baccano!',
 'Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.',
 'Fate/Zero 2nd Season',
 'Shinsekai yori',
 'One Punch Man',
 'Sen to Chi

In [25]:
df2 = df[['user_id', 'anime_id', 'rating']]

In [26]:
df2.head()

,user_id,anime_id,rating
0,1,20,0
1,3,20,8
2,5,20,6
3,6,20,0
4,10,20,0


In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813720 entries, 0 to 7813719
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


# Membuat Model

In [28]:
# Membaca data ke dalam format Surprise
reader = Reader(rating_scale=(0, 10))
dataset = Dataset.load_from_df(pd.DataFrame(df2), reader)

In [29]:
# Membangun model SVD
model = SVD()
trainset = dataset.build_full_trainset()
model.fit(trainset)

In [30]:
# Melakukan validasi silang
cross_validate(model, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9391  1.9377  1.9378  1.9392  1.9368  1.9381  0.0009  
MAE (testset)     1.2387  1.2367  1.2376  1.2374  1.2370  1.2375  0.0007  
Fit time          108.52  109.91  106.38  123.56  121.55  113.99  7.12    
Test time         23.09   18.73   22.48   22.69   18.42   21.08   2.06    


{'test_rmse': array([1.93906888, 1.93766053, 1.93776807, 1.93915465, 1.93675271]),
 'test_mae': array([1.23867997, 1.23665587, 1.23757985, 1.23741452, 1.23701565]),
 'fit_time': (108.52000975608826,
  109.91381192207336,
  106.380690574646,
  123.5644097328186,
  121.55320906639099),
 'test_time': (23.094053268432617,
  18.725700855255127,
  22.482516050338745,
  22.685811758041382,
  18.418908834457397)}

In [31]:
# Prediksi rating user tertentu terhadap anime_id tertentu
user_id = 50
anime_id = 717
predicted_rating = model.predict(user_id, anime_id)

In [32]:
print(f"Predicted rating for user {user_id} on anime {anime_id}: {predicted_rating.est}")

Predicted rating for user 50 on anime 717: 6.578846512697383
